# Camera pose 0, 0

In [45]:
import plotly.graph_objects as go
import numpy as np
import torch
import plotly.express as px

# Ray Casting

In [55]:
# 0.
def to_pytorch(tensor, return_type=False):
    ''' Converts input tensor to pytorch.
    Args:
        tensor (tensor): Numpy or Pytorch tensor
        return_type (bool): whether to return input type
    '''
    is_numpy = False
    if type(tensor) == np.ndarray:
        tensor = torch.from_numpy(tensor).float()
        is_numpy = True
    tensor = tensor.clone()
    if return_type:
        return tensor, is_numpy
    return tensor

# 1. get camera intrinsic
def get_camera_mat(fov=49.13, invert=True):
    # fov = 2 * arctan( sensor / (2 * focal))
    # focal = (sensor / 2)  * 1 / (tan(0.5 * fov))
    # in our case, sensor = 2 as pixels are in [-1, 1]
    focal = 1. / np.tan(0.5 * fov * np.pi/180.)
    focal = focal.astype(np.float32)
    mat = torch.tensor([
        [focal, 0., 0., 0.],
        [0., focal, 0., 0.],
        [0., 0., 1, 0.],
        [0., 0., 0., 1.]
    ]).reshape(1, 4, 4)

    if invert:
        mat = torch.inverse(mat)
    return mat

# 2. get camera position with camera pose (theta & phi)
def to_sphere(u, v):
    theta = 2 * np.pi * u
    phi = np.arccos(1 - 2 * v)
    cx = np.sin(phi) * np.cos(theta)
    cy = np.sin(phi) * np.sin(theta)
    cz = np.cos(phi)
    return np.stack([cx, cy, cz], axis=-1)

# 3. get camera coordinate system assuming it points to the center of the sphere
def look_at(eye, at=np.array([0, 0, 0]), up=np.array([0, 0, 1]), eps=1e-5,
            to_pytorch=True):
    at = at.astype(float).reshape(1, 3)
    up = up.astype(float).reshape(1, 3)
    eye = eye.reshape(-1, 3)
    up = up.repeat(eye.shape[0] // up.shape[0], axis=0)
    eps = np.array([eps]).reshape(1, 1).repeat(up.shape[0], axis=0)

    z_axis = eye - at
    z_axis /= np.max(np.stack([np.linalg.norm(z_axis,
                                              axis=1, keepdims=True), eps]))

    x_axis = np.cross(up, z_axis)
    x_axis /= np.max(np.stack([np.linalg.norm(x_axis,
                                              axis=1, keepdims=True), eps]))

    y_axis = np.cross(z_axis, x_axis)
    y_axis /= np.max(np.stack([np.linalg.norm(y_axis,
                                              axis=1, keepdims=True), eps]))

    r_mat = np.concatenate(
        (x_axis.reshape(-1, 3, 1), y_axis.reshape(-1, 3, 1), z_axis.reshape(
            -1, 3, 1)), axis=2)

    if to_pytorch:
        r_mat = torch.tensor(r_mat).float()

    return r_mat

# 5. arange 2d array of pixel coordinate and give depth of 1
def arange_pixels(resolution=(128, 128), batch_size=1, image_range=(-1., 1.),
                  subsample_to=None, invert_y_axis=False):
    ''' Arranges pixels for given resolution in range image_range.
    The function returns the unscaled pixel locations as integers and the
    scaled float values.
    Args:
        resolution (tuple): image resolution
        batch_size (int): batch size
        image_range (tuple): range of output points (default [-1, 1])
        subsample_to (int): if integer and > 0, the points are randomly
            subsampled to this value
    '''
    h, w = resolution
    n_points = resolution[0] * resolution[1]

    # Arrange pixel location in scale resolution
    pixel_locations = torch.meshgrid(torch.arange(0, w), torch.arange(0, h))
    pixel_locations = torch.stack(
        [pixel_locations[0], pixel_locations[1]],
        dim=-1).long().view(1, -1, 2).repeat(batch_size, 1, 1)
    pixel_scaled = pixel_locations.clone().float()

    # Shift and scale points to match image_range
    scale = (image_range[1] - image_range[0])
    loc = scale / 2
    pixel_scaled[:, :, 0] = scale * pixel_scaled[:, :, 0] / (w - 1) - loc
    pixel_scaled[:, :, 1] = scale * pixel_scaled[:, :, 1] / (h - 1) - loc

    # Subsample points if subsample_to is not None and > 0
    if (subsample_to is not None and subsample_to > 0 and
            subsample_to < n_points):
        idx = np.random.choice(pixel_scaled.shape[1], size=(subsample_to,),
                               replace=False)
        pixel_scaled = pixel_scaled[:, idx]
        pixel_locations = pixel_locations[:, idx]

    if invert_y_axis:
        assert(image_range == (-1, 1))
        pixel_scaled[..., -1] *= -1.
        pixel_locations[..., -1] = (h - 1) - pixel_locations[..., -1]

    return pixel_locations, pixel_scaled

# 6. mat_mul with intrinsic and then extrinsic gives you p_world (pixels in world) 
def image_points_to_world(image_points, camera_mat, world_mat, scale_mat=None,
                          invert=False, negative_depth=True):
    ''' Transforms points on image plane to world coordinates.
    In contrast to transform_to_world, no depth value is needed as points on
    the image plane have a fixed depth of 1.
    Args:
        image_points (tensor): image points tensor of size B x N x 2
        camera_mat (tensor): camera matrix
        world_mat (tensor): world matrix
        scale_mat (tensor): scale matrix
        invert (bool): whether to invert matrices (default: False)
    '''
    batch_size, n_pts, dim = image_points.shape
    assert(dim == 2)
    d_image = torch.ones(batch_size, n_pts, 1)
    if negative_depth:
        d_image *= -1.
    return transform_to_world(image_points, d_image, camera_mat, world_mat,
                              scale_mat, invert=invert)

def transform_to_world(pixels, depth, camera_mat, world_mat, scale_mat=None,
                       invert=True, use_absolute_depth=True):
    ''' Transforms pixel positions p with given depth value d to world coordinates.
    Args:
        pixels (tensor): pixel tensor of size B x N x 2
        depth (tensor): depth tensor of size B x N x 1
        camera_mat (tensor): camera matrix
        world_mat (tensor): world matrix
        scale_mat (tensor): scale matrix
        invert (bool): whether to invert matrices (default: true)
    '''
    assert(pixels.shape[-1] == 2)

    if scale_mat is None:
        scale_mat = torch.eye(4).unsqueeze(0).repeat(
            camera_mat.shape[0], 1, 1)

    # Convert to pytorch
    pixels, is_numpy = to_pytorch(pixels, True)
    depth = to_pytorch(depth)
    camera_mat = to_pytorch(camera_mat)
    world_mat = to_pytorch(world_mat)
    scale_mat = to_pytorch(scale_mat)

    # Invert camera matrices
    if invert:
        camera_mat = torch.inverse(camera_mat)
        world_mat = torch.inverse(world_mat)
        scale_mat = torch.inverse(scale_mat)

    # Transform pixels to homogen coordinates
    pixels = pixels.permute(0, 2, 1)
    pixels = torch.cat([pixels, torch.ones_like(pixels)], dim=1)

    # Project pixels into camera space
    if use_absolute_depth:
        pixels[:, :2] = pixels[:, :2] * depth.permute(0, 2, 1).abs()
        pixels[:, 2:3] = pixels[:, 2:3] * depth.permute(0, 2, 1)
    else:
        pixels[:, :3] = pixels[:, :3] * depth.permute(0, 2, 1)
        
    # Transform pixels to world space
    p_world = scale_mat @ world_mat @ camera_mat @ pixels

    # Transform p_world back to 3D coordinates
    p_world = p_world[:, :3].permute(0, 2, 1)

    if is_numpy:
        p_world = p_world.numpy()
    return p_world


# 7. mat_mul zeros with intrinsic&extrinsic for camera pos (which we alread obtained as loc)
def origin_to_world(n_points, camera_mat, world_mat, scale_mat=None,
                    invert=False):
    ''' Transforms origin (camera location) to world coordinates.
    Args:
        n_points (int): how often the transformed origin is repeated in the
            form (batch_size, n_points, 3)
        camera_mat (tensor): camera matrix
        world_mat (tensor): world matrix
        scale_mat (tensor): scale matrix
        invert (bool): whether to invert the matrices (default: false)
    '''
    
    batch_size = camera_mat.shape[0]
    device = camera_mat.device
    # Create origin in homogen coordinates
    p = torch.zeros(batch_size, 4, n_points).to(device)
    p[:, -1] = 1.

    if scale_mat is None:
        scale_mat = torch.eye(4).unsqueeze(
            0).repeat(batch_size, 1, 1).to(device)

    # Invert matrices
    if invert:
        camera_mat = torch.inverse(camera_mat)
        world_mat = torch.inverse(world_mat)
        scale_mat = torch.inverse(scale_mat)
        
    camera_mat = to_pytorch(camera_mat)
    world_mat = to_pytorch(world_mat)
    scale_mat = to_pytorch(scale_mat)
    
    # Apply transformation
    p_world = scale_mat @ world_mat @ camera_mat @ p

    # Transform points back to 3D coordinates
    p_world = p_world[:, :3].permute(0, 2, 1)
    return p_world


In [63]:
u = 1
v = 0.5
depth_range=[0.5, 6.]
n_ray_samples=16
resolution_vol = 4
batch_size = 1

res = resolution_vol
n_points = res * res

# 1. get camera intrinsic 
camera_mat = get_camera_mat()

# 2. get camera position with camera pose (theta & phi)
loc = to_sphere(u, v)

# 3. get camera coordinate system assuming it points to the center of the sphere
R = look_at(loc)

# 4. The carmera coordinate is the rotational matrix and with camera loc, it is camera extrinsic
RT = np.eye(4).reshape(1, 4, 4)
RT[:, :3, :3] = R
RT[:, :3, -1] = loc
world_mat = RT

# 5. arange 2d array of pixel coordinate and give depth of 1
pixels = arange_pixels((res, res), 1, invert_y_axis=False)[1]
pixels[..., -1] *= -1. # still dunno why this is here

# 6. mat_mul with intrinsic and then extrinsic gives you p_world (pixels in world) 
pixels_world = image_points_to_world(pixels, camera_mat, world_mat)

# 7. mat_mul zeros with intrinsic&extrinsic for camera pos (which we alread obtained as loc)
camera_world = origin_to_world(n_points, camera_mat, world_mat)

# 8. ray = pixel - camera origin (in world)
ray_vector = pixels_world - camera_world

# 9. depths from closest to furthest (0.5 ~ 6.0)
di = depth_range[0] + \
    torch.linspace(0., 1., steps=n_ray_samples).reshape(1, 1, -1) * (
        depth_range[1] - depth_range[0])
di = di.repeat(batch_size, n_points, 1)

# 10. 
p_i = camera_world.unsqueeze(-2).contiguous() + \
    di.unsqueeze(-1).contiguous() * ray_vector.unsqueeze(-2).contiguous()

# Visualization

In [42]:
# draw sphere with radius r 
# also draw contours and vertical lines
def draw_sphere(r=1):
    # sphere
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = r * np.outer(np.cos(u), np.sin(v))
    y = r * np.outer(np.sin(u), np.sin(v))
    z = r * np.outer(np.ones(np.size(u)), np.cos(v))
    
    # vertical lines on sphere
    u2 = np.linspace(0, 2 * np.pi, 20)
    x2 = r * np.outer(np.cos(u2), np.sin(v))
    y2 = r * np.outer(np.sin(u2), np.sin(v))
    z2 = r * np.outer(np.ones(np.size(u2)), np.cos(v))
    
    # create sphere and draw sphere with contours
    fig = go.Figure(data=[go.Surface(x=x, y=y, z=z, 
                                 colorscale='Greys', opacity=0.2,
                                 contours = {
                                     'z' : {'show' : True, 'start' : -1,
                                           'end' : 1, 'size' : 0.1,
                                           'color' : 'white',
                                           'width' : 1}
                                 }
                                , showscale=False)])
    
    # vertical lines on sphere
    for i in range(len(u2)):
        fig.add_scatter3d(x=x2[i], y=y2[i], z=z2[i], 
                          line=dict(
                              color='white',
                              width=1
                          ),
                         mode='lines',
                         showlegend=False)
    
    return fig

# draw xyplane
def draw_XYplane(fig, x_range = [-2, 2], y_range = [-2, 2]):
    x3 = np.linspace(x_range[0], x_range[1], 100)
    y3 = np.linspace(y_range[0], y_range[1], 100)
    z3 = np.zeros(shape=(100,100))
    
    fig.add_surface(x=x3, y=y3, z=z3,
                colorscale ='Greys', opacity=0.8,
                showscale=False
    )
    
    return fig
    

def draw_XYZworld(fig, size=1.5):
    # x, y, z positive direction (world)
    X_axis = [0, size]
    X_text = [None, "X"]
    X0 = [0, 0]
    Y_axis = [0, size]
    Y_text = [None, "Y"]
    Y0 = [0, 0]
    Z_axis = [0, size]
    Z_text = [None, "Z"]
    Z0 = [0, 0]
    
    fig.add_scatter3d(x=X_axis, y=Y0, z=Z0, 
                      line=dict(
                          color='red',
                          width=10
                      ),
                    mode='lines+text',
                    text=X_text,
                    textposition='top center',
                    textfont=dict(
                        color="red",
                        size=18
                    ),
                    showlegend=False)

    fig.add_scatter3d(x=X0, y=Y_axis, z=Z0, 
                          line=dict(
                              color='green',
                              width=10
                          ),
                         mode='lines+text',
                        text=Y_text,
                        textposition='top center',
                        textfont=dict(
                            color="green",
                            size=18
                        ),
                        showlegend=False)

    fig.add_scatter3d(x=X0, y=Y0, z=Z_axis, 
                          line=dict(
                              color='blue',
                              width=10
                          ),
                         mode='lines+text',
                        text=Z_text,
                        textposition='top center',
                        textfont=dict(
                            color="blue",
                            size=18
                        ),
                        showlegend=False)
    
    return fig

# draw cam and cam coordinate system
def draw_cam_init(fig, pos=[0, -1, 0], size=0.3):
    # camera at init
    Xc = [0]
    Yc = [-1]
    Zc = [0]
    c_text = "Camera"

    # camera axis
    Xc_axis = [0, 0.3]
    Xc_text = [None, "Xc"]
    Xc0 = [0, 0]

    # -z in world perspective
    Yc_axis = [-1, -0.7]
    Yc_text = [None, "Zc"]
    Yc0 = [-1, -1]

    # y in world perspective
    Zc_axis = [0, -0.3]
    Zc_text = [None, "Yc"]
    Zc0 = [0, 0]
    
    # cam pos
    fig.add_scatter3d(x=Xc, y=Yc, z=Zc, 
                     mode='markers',
                  marker=dict(
                      color='yellow',
                      size=2,
                      sizemode='diameter'
                  ),
                    showlegend=False)

    # camera axis
    fig.add_scatter3d(x=Xc_axis, y=Yc0, z=Zc0, 
                          line=dict(
                              color='red',
                              width=10
                          ),
                        mode='lines+text',
                        text=Xc_text,
                        textposition='top center',
                        textfont=dict(
                            color="red",
                            size=18
                        ),
                        showlegend=False)

    fig.add_scatter3d(x=Xc0, y=Yc_axis, z=Zc0, 
                          line=dict(
                              color='green',
                              width=10
                          ),
                         mode='lines+text',
                        text=Yc_text,
                        textposition='top center',
                        textfont=dict(
                            color="green",
                            size=18
                        ),
                        showlegend=False)

    fig.add_scatter3d(x=Xc0, y=Yc0, z=Zc_axis, 
                          line=dict(
                              color='blue',
                              width=10
                          ),
                         mode='lines+text',
                        text=Zc_text,
                        textposition='top center',
                        textfont=dict(
                            color="blue",
                            size=18
                        ),
                        showlegend=False)
    
    return fig


# draw near&far frustrum with rays connecting the corners
def draw_ray_frus(fig):
    
    # camera at init
    Xc = [0]
    Yc = [-1]
    Zc = [0]
    
    Xfrus_near = [0.3, 0.3, -0.3, -0.3, 0.3]
    Xfrus_far = [1.2, 1.2, -1.2, -1.2, 1.2]

    Zfrus_near = [-0.3, 0.3, 0.3, -0.3, -0.3]
    Zfrus_far = [-1.2, 1.2, 1.2, -1.2, -1.2]

    Yfrus_near = [-0.5, -0.5, -0.5, -0.5, -0.5]
    Yfrus_far = [1.5, 1.5, 1.5, 1.5, 1.5]

    # ray connecting corners of the frustrum
    Xray0 = Xc + [Xfrus_near[0], Xfrus_far[0]]
    Xray1 = Xc + [Xfrus_near[1], Xfrus_far[1]]
    Xray2 = Xc + [Xfrus_near[2], Xfrus_far[2]]
    Xray3 = Xc + [Xfrus_near[3], Xfrus_far[3]]

    Yray0 = Yc + [Yfrus_near[0], Yfrus_far[0]]
    Yray1 = Yc + [Yfrus_near[1], Yfrus_far[1]]
    Yray2 = Yc + [Yfrus_near[2], Yfrus_far[2]]
    Yray3 = Yc + [Yfrus_near[3], Yfrus_far[3]]

    Zray0 = Zc + [Zfrus_near[0], Zfrus_far[0]]
    Zray1 = Zc + [Zfrus_near[1], Zfrus_far[1]]
    Zray2 = Zc + [Zfrus_near[2], Zfrus_far[2]]
    Zray3 = Zc + [Zfrus_near[3], Zfrus_far[3]]
    
    # near/far frustrum
    fig.add_scatter3d(x=Xfrus_near, y=Yfrus_near, z=Zfrus_near, 
                          line=dict(
                              color='yellow',
                              width=5
                          ),
                         mode='lines',
                        showlegend=False)

    fig.add_scatter3d(x=Xfrus_far, y=Yfrus_far, z=Zfrus_far, 
                          line=dict(
                              color='yellow',
                              width=5
                          ),
                         mode='lines',
                        showlegend=False)


    # ray connecting corners of the frustrum
    fig.add_scatter3d(x=Xray0, y=Yray0, z=Zray0, 
                          line=dict(
                              color='yellow',
                              width=5
                          ),
                         mode='lines',
                        showlegend=False)

    fig.add_scatter3d(x=Xray1, y=Yray1, z=Zray1, 
                          line=dict(
                              color='yellow',
                              width=5
                          ),
                         mode='lines',
                        showlegend=False)

    fig.add_scatter3d(x=Xray2, y=Yray2, z=Zray2, 
                          line=dict(
                              color='yellow',
                              width=5
                          ),
                         mode='lines',
                        showlegend=False)

    fig.add_scatter3d(x=Xray3, y=Yray3, z=Zray3, 
                          line=dict(
                              color='yellow',
                              width=5
                          ),
                         mode='lines',
                        showlegend=False)
    
    return fig

In [44]:
# sphere
fig = draw_sphere(r = 1)

# change figure size
fig.update_layout(autosize=False, width = 1000, height=1000)

# xy plane
fig = draw_XYplane(fig)

# show world coordinate system (X, Y, Z positive direction)
fig = draw_XYZworld(fig)

#  draw camera at init (with its cooridnate system)
fig = draw_cam_init(fig)

# draw near&far frustrum with rays connecting the corners
fig = draw_ray_frus(fig)

fig.show()